In [ ]:
def overwrite_ref_config(ref_config, run_info):
    
    '''
    This is a workaround for adjusting the configuration of those runs that didn't have a YAML configuration file logged as an artifact.
    '''
    
    config = deepcopy(ref_config)
    config.network_architecture.latent_dim = int(run_info["params.latent_dim"])
    config.loss.regularization.weight = float(run_info["params.w_kl"])
    config.optimizer.parameters.lr = float(run_info["params.lr"])
    config.sample_sizes = [100, 100, 100, 100]
    
    return config

pl.utilities.seed.reset_seed() # seed_everything(seed=None)
pl.utilities.seed.seed_everything(seed=None)

ref_config = load_yaml_config("config_files/config.yaml")
config = overwrite_ref_config(ref_config, run_info)
pprint(to_dict(config))

dm = get_datamodule(config, perform_setup=True)
model = get_lightning_module(config, dm)

weights = get_model_pretrained_weights(runs_df, exp_id, run_id)
model.model.load_state_dict(_model_pretrained_weights)

In [ ]:
s = dm.dataset[1]['s']
s_hat = model(s)[0][0]
mse(s, s_hat)